In [1]:
import pennylane as qml
from pennylane import numpy as np

### Simulating Dynamics of Tri-Junction

### 1. From Beenakker's Lecture note

In [18]:
d_max, d_min = 0.99, 0.01
tau = 1
N = 100
dt = tau / N

In [19]:
def Coupling_strength_cube(t):
    if t == 0:
        d1, d2, d3 = d_min, d_min, d_max
    elif t < tau:
        d1 = d_min + (d_max - d_min) * t / tau
        d2, d3 = d_min, d_max
    elif t < 2 * tau:
        d1, d2 = d_max, d_min
        d3 = d_max - (d_max - d_min) * (t - tau) / tau
    elif t < 3 * tau:
        d1, d3 = d_max, d_min
        d2 = d_min + (d_max - d_min) * (t - 2 * tau) / tau
    elif t < 4 * tau:
        d2, d3 = d_max, d_min
        d1 = d_max - (d_max - d_min) * (t - 3 * tau) / tau
    elif t < 5 * tau:
        d1, d2 = d_min, d_max
        d3 = d_min + (d_max - d_min) * (t - 4 * tau) / tau
    elif t < 6 * tau:
        d1, d3 = d_min, d_max
        d2 = d_max - (d_max - d_min) * (t - 5 * tau) / tau
    elif t == 6 * tau:
        d1, d2, d3 = d_min, d_min, d_max
    return d1, d2, d3

def Coupling_strength_tetrahedron(t):
    if t == 0:
        d1, d2, d3 = d_min, d_min, d_max
    elif t < tau:
        d2 = d_min
        d1 = d_min + (d_max - d_min) * t / tau
        d3 = d_max - (d_max - d_min) * t / tau
    elif t < 2 * tau:
        d3 = d_min
        d1 = d_max - (d_max - d_min) * (t - tau) / tau
        d2 = d_min + (d_max - d_min) * (t - tau) / tau
    elif t < 3 * tau:
        d1 = d_min
        d2 = d_max - (d_max - d_min) * (t - 2 * tau) / tau
        d3 = d_min + (d_max - d_min) * (t - 2 * tau) / tau
    elif t == 3 * tau:
        d1, d2, d3 = d_min, d_min, d_max
    return d1, d2, d3
    
def H(t, parameter_path):
    if parameter_path == 'cube':
        d1, d2, d3 = Coupling_strength_cube(t)
    elif parameter_path == 'tetrahedron':
        d1, d2, d3 = Coupling_strength_tetrahedron(t)
    H = qml.Hamiltonian(
        [-d3, -d2, d1],
        [qml.Identity(0) @ qml.PauliZ(1), qml.PauliX(0) @ qml.PauliX(1), qml.PauliY(0) @ qml.PauliX(1)])
    return H

In [20]:
def Evolution_dt(t, parameter_path):
    Hamiltonian = H(t, parameter_path)
    qml.templates.ApproxTimeEvolution(Hamiltonian, dt, 1)
    
def initialize(state, ):
    d1, d2, d3 = d_min, d_min, d_max
    d = (d1**2 + d2**2 + d3**2)**(1/2)
    if state == 'plus':
        statevector = np.array([1j * (d + d3), 0, 0, d1 + 1j * d2])
        statevector = statevector / np.linalg.norm(statevector)
        print("Initial State is: " + str(statevector))
    elif state == 'minus':
        statevector = np.array([0, 1j * (d - d3), d1 + 1j * d2, 0])
        statevector = statevector / np.linalg.norm(statevector)
        print("Initial State is: " + str(statevector))
    else:
        print("Non-valid initial state")
    qml.templates.AmplitudeEmbedding(statevector, wires=[0,1], normalize=True)

In [26]:
dev1 = qml.device('default.qubit', wires=2)

@qml.qnode(dev1)
def circuit(parameter_path):
    if parameter_path == 'cube':
        period_num = 6
    elif parameter_path == 'tetrahedron':
        period_num = 3
        
    initialize('minus')
    
  
    t = 0
    for i in range(period_num):
        for i in range(N):
            #print("t: " + str(t))
            Evolution_dt(t, parameter_path)
            t = t + dt  
    return qml.state()

In [27]:
circuit('cube')

Initial State is: [0.        +0.j         0.        +0.00714195j 0.70708875+0.70708875j
 0.        +0.j        ]


tensor([-6.10622664e-16+3.65046806e-15j,  9.76088856e-02-2.16215915e-04j,
        -7.65766254e-01+6.35668548e-01j, -7.21644966e-16-8.88178420e-16j], requires_grad=True)

In [ ]:
J_max = 1
a = J_max * 3
tau = 3 / J_max 

N = 3 # Suzuki-Trotter Step
dt = tau / N

In [ ]:
dev = qml.device('default.qubit', wires=3, shots=1000)

In [ ]:
# define the Hamiltonian
def J(t):
    if t < tau:
        J_01 = J_max - J_max * t / tau
        J_12 = J_max * t / tau
        J_20 = 0
    elif t < 2 * tau:
        J_01 = 0
        J_12 = J_max - J_max * (t - tau) / tau
        J_20 = J_max * (t - tau) / tau
    elif t < 3 * tau:
        J_01 = J_max * (t - 2 * tau) / tau
        J_12 = 0
        J_20 = J_max - J_max * (t - 2 * tau) / tau
    elif t < 4 * tau:
        J_01 = J_max - J_max * (t - 3 * tau) / tau
        J_12 = J_max * (t - 3 * tau) / tau
        J_20 = 0
    elif t < 5 * tau:
        J_01 = 0
        J_12 = J_max - J_max * (t - 4 * tau) / tau
        J_20 = J_max * (t - 4 * tau) / tau
    elif t < 6 * tau:
        J_01 = J_max * (t - 5 * tau) / tau
        J_12 = 0
        J_20 = J_max - J_max * (t - 5 * tau) / tau
    
    J_list = [J_01, J_12, J_20]
    for i in range(3):
        if J_list[i] < 1e-10:
            J_list[i] = 0
    return J_list[0], J_list[1], J_list[2]

def Hamiltonian(t):
    J_01, J_12, J_20 = J(t)
    H = qml.Hamiltonian(
        [a, a, a, J_01, J_12, J_20],
        [qml.PauliZ(0), qml.PauliZ(1), qml.PauliZ(2), qml.PauliY(0) @ qml.PauliX(1),
        qml.PauliY(1) @ qml.PauliX(2), qml.PauliY(0) @ qml.PauliZ(1) @ qml.PauliX(2)])
    return H

In [ ]:
def Initialize(state):
    qml.RX(-1 * np.pi / 2 , wires=[0])
    qml.CNOT(wires=[0,1])
    if state == 'plus':
        qml.RY(-1 * np.pi / 2, wires=[0])
    elif state == 'minus':
        qml.RY(np.pi / 2, wires=[0])
    qml.RY(np.pi, wires=[2])
    
def Initialize_inverse(state):
    qml.RY(4 * np.pi - np.pi, wires=[2])
    if state == 'plus':
        qml.RY(4 * np.pi + 1 * np.pi / 2, wires=[0])
    elif state == 'minus':
        qml.RY(4 * np.pi - np.pi / 2, wires=[0])
    qml.CNOT(wires=[0,1])
    qml.RX(4 * np.pi + 1 * np.pi / 2 , wires=[0])
    
def Evolution_dt(t):
    H = Hamiltonian(t)
    qml.templates.ApproxTimeEvolution(H, dt, 5)
    
def Evolution_dt_alternative1(t):

In [ ]:
def Evolution_dt_alternative(t):
    J_01, J_12, J_20 = J(t)
    
    qml.RZ(a * dt, wires=[0])
    qml.RZ(a * dt, wires=[1])
    qml.RZ(a * dt, wires=[2])
    
    qml.CNOT(wires=[0,1])
    qml.RY(J_01 * dt, wires=[0])
    qml.CNOT(wires=[0,1])
    
    qml.CNOT(wires=[1,2])
    qml.RY(J_12 * dt, wires=[1])
    qml.CNOT(wires=[1,2])
    
    qml.CNOT(wires=[0,2])
    qml.CZ(wires=[0,1])
    qml.RY(J_20 * dt, wires=[0])
    qml.CZ(wires=[0,1])
    qml.CNOT(wires=[0,2])

In [ ]:
@qml.qnode(dev)
def circuit():
    # Initialize + state
    Initialize('plus')
    
    # Time Evolution
    t = 0
    for j in range(6):
        for i in range(N):
            print(t)
            Evolution_dt_alternative(t)
            t = t + dt
    
    Initialize_inverse('minus')
    return qml.state()

In [ ]:
circuit()